# Downloading data from Google Drive

In [ ]:
!pip install gdown


In [ ]:
#import gdown
# to download the data from Google Drive
# id = 1XKcjj7eEAmgmbHU4Qa7g2fUbHHXRkCBi   ( saber_11_pro.csv)
#url = 'https://drive.google.com/uc?export=download&id=1XKcjj7eEAmgmbHU4Qa7g2fUbHHXRkCBi'
#output = 'saber_11_pro.csv'
#gdown.download(url, output, quiet=False)


In [ ]:
#import gdown
# to download the data from Google Drive
# id = 10OhmdA_gI3EDYj0IS0uUKsYDot_IBUnu    (saber_combined_all_fields.csv)
#url ='https://drive.google.com/uc?export=download&id=10OhmdA_gI3EDYj0IS0uUKsYDot_IBUnu'
#output = 'saber_combined_all_fields.csv'
#gdown.download(url, output, quiet=False)

In [ ]:
#import gdown
# to download the data from Google Drive
# id = 1HOBcz5K_QfeYavdDl2iCkJGAhlh1SSJV    (saber_combined_all_fields.csv)
#url ='https://drive.google.com/uc?export=download&id=1HOBcz5K_QfeYavdDl2iCkJGAhlh1SSJV'
#output = 'saber_combined_preprocessed.csv'
#gdown.download(url, output, quiet=False)

# Libraries

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

import plotly.express as px
import plotly.graph_objs as go

from sklearn import preprocessing

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score,  r2_score,  f1_score

 
from sklearn import model_selection  # Kfold..
# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

import warnings; 
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500) # to show more rows.




# DF, Dataset : Saber_Combined_Preprocessed

In [2]:
df = pd.read_csv('../input/k/ayoubberdeddouch/colombia-carreer-recosystem/saber_combined_preprocessed.csv')
df.head()

In [3]:
df.info()

In [4]:
df.columns

In [8]:
def nan_check(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
    return missing_data

nan_check(df)

In [5]:
df.describe().T.style.bar(subset=['mean'], color='#FF595E')\
                           .background_gradient(subset=['50%'], cmap='PiYG') # highlight median

In [6]:
# Categorical features
cat_col=df.select_dtypes(include='object').columns.to_list()
cat_col, len(cat_col)


In [7]:
# Numerical features
num_col=df.select_dtypes(include='number').columns.to_list()
num_col, len(num_col)

In [9]:
%%time
### Corralation matrix heatmap
# Getting correlation matrix
cor_matrix = df[num_col].corr().round(1)
mask = np.triu(np.ones_like(cor_matrix, dtype = bool))
# Plotting heatmap 
fig = plt.figure(figsize=(25,25));
plt.title('Correlation matrix for df Saber Combined PreProcessed')
sns.heatmap(cor_matrix, mask=mask, annot=True, center=0,linewidths = .5,square=True,cbar_kws={"shrink": .60}, cmap = sns.diverging_palette(250, 10, as_cmap=True), ax=plt.subplot(111));
plt.show()

# Building the Recommendation System

In [10]:
recsys_df = df.copy()
recsys_df.head()

In [11]:
# Categorical features
cat_col=recsys_df.select_dtypes(include='object').columns.to_list()
cat_col, len(cat_col)


In [13]:
#%%time
#[' Column : {} with unique: {} --- \n'.format(col,recsys_df[col ].unique()) for col in cat_col]

In [ ]:
lst = recsys_df.your_municipality_resides.value_counts()
vals = recsys_df.your_municipality_resides.unique()
df1 = pd.DataFrame(columns=['Count'],index= vals)
df1.Count = lst
df1 = df1.sort_values(by='Count', ascending=False)
### plot bar 
fig = px.bar(df1, x =df1.index , y ="Count" ,title="Municipality Residence",
              color_continuous_scale ='RdBu',color="Count",height=500)
fig.update_xaxes(tickangle=45)
fig.show()



In [ ]:

lst = recsys_df.inst_institution_name.value_counts()
vals = recsys_df.inst_institution_name.unique()
df2 = pd.DataFrame(columns=['Count'],index= vals)
df2.Count = lst
df2 = df2.sort_values(by='Count', ascending=False)
### plot bar 
fig = px.bar(df2, x =df2.index , y ="Count" ,title="Institution Name",
              color_continuous_scale ='RdBu',color="Count",height=500)
fig.update_xaxes(tickangle=45)
fig.show()



In [ ]:

lst = recsys_df.your_inst_municipality.value_counts()
vals = recsys_df.your_inst_municipality.unique()
df3 = pd.DataFrame(columns=['Count'],index= vals)
df3.Count = lst
df3 = df3.sort_values(by='Count', ascending=False)
### plot bar 
fig = px.bar(df3, x =df3.index , y ="Count" ,title="Institution Municipality Name",
              color_continuous_scale ='RdBu',color="Count",height=500)
fig.update_xaxes(tickangle=45)
fig.show()



These 2 values are the same I think.
**'BOGOTÁ D.C.' == 'BOGOTÁ,**

In [ ]:
lst = recsys_df.score_english_saber_11_category.value_counts()
vals = recsys_df.score_english_saber_11_category.unique()
df3 = pd.DataFrame(columns=['Count'],index= vals)
df3.Count = lst
df3 = df3.sort_values(by='Count', ascending=False)
### plot bar 
fig = px.bar(df3, x =df3.index , y ="Count" ,title="Score English Saber_11 Category",
              color_continuous_scale ='RdBu',color="Count",height=500)
fig.update_xaxes(tickangle=45)
fig.show()



In [ ]:

lst = recsys_df.your_undergraduate_core.value_counts()
vals = recsys_df.your_undergraduate_core.unique()
df3 = pd.DataFrame(columns=['Count'],index= vals)
df3.Count = lst
df3 = df3.sort_values(by='Count', ascending=False)
### plot bar 
fig = px.bar(df3, x =df3.index , y ="Count" ,title="Your_undergraduate_core",
              color_continuous_scale ='RdBu',color="Count",height=500)
fig.update_xaxes(tickangle=45)
fig.show()



In [ ]:

lst = recsys_df.cole_name_campus.value_counts()
vals = recsys_df.cole_name_campus.unique()
df3 = pd.DataFrame(columns=['Count'],index= vals)
df3.Count = lst
df3 = df3.sort_values(by='Count', ascending=False)
### plot bar 
fig = px.bar(df3, x =df3.index , y ="Count" ,title="College Name Campus",
              color_continuous_scale ='RdBu',color="Count",height=500)
fig.update_xaxes(tickangle=45)
fig.show()



In [ ]:
# Numerical features
num_col=recsys_df.select_dtypes(include='number').columns.to_list()
num_col, len(num_col)

In [17]:
features = [
   'score_math_saber_pro',
    'score_language_saber_pro',
    'score_social_science_saber_pro',
    'score_english_saber_pro',
    'score_writing_saber_pro',
    ## 
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
]

In [18]:
recsys_df[features].head()

In [19]:
recsys_df[features].describe().T.style.bar(subset=['mean'], color='#FF595E')\
                           .background_gradient(subset=['50%'], cmap='PiYG') # highlight median

In [34]:
%%time
### Corralation matrix heatmap
# Getting correlation matrix
cor_matrix = recsys_df[features].corr().round(1)
mask = np.triu(np.ones_like(cor_matrix, dtype = bool))
# Plotting heatmap 
fig = plt.figure(figsize=(25,25));
plt.title('Correlation matrix for df Saber Combined PreProcessed')
sns.heatmap(cor_matrix, mask=mask, annot=True, center=0,linewidths = .5,square=True,cbar_kws={"shrink": .60}, cmap = sns.diverging_palette(250, 10, as_cmap=True), ax=plt.subplot(111));
plt.show()

These features are all Correlated 0.9
* 'score_math_saber_pro',
*  'score_language_saber_pro',
*  'score_social_science_saber_pro',
*  'score_english_saber_pro',
* 'score_writing_saber_pro',

In [21]:
features = [
    ## 
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
]

In [35]:
%%time
### Corralation matrix heatmap
# Getting correlation matrix
cor_matrix = recsys_df[features].corr().round(1)
mask = np.triu(np.ones_like(cor_matrix, dtype = bool))
# Plotting heatmap 
fig = plt.figure(figsize=(25,25));
plt.title('Correlation matrix for df Saber Combined PreProcessed')
sns.heatmap(cor_matrix, mask=mask, annot=True, center=0,linewidths = .5,square=True,cbar_kws={"shrink": .60}, cmap = sns.diverging_palette(250, 10, as_cmap=True), ax=plt.subplot(111));
plt.show()

In [36]:
for var in features:
    print(var.upper() + ' Distribution')
    print(recsys_df[var].describe())
    print('-----')

Note that the max-min range of all 3 variables are very different from one another. When we do further analysis, like multivariate linear regression, for example, the attributed income will intrinsically influence the result more due to its larger value. Hence, it is important to standardize and normalize the data to bring all variables to the same range.

I use a Robust Scaler (QuantileTransformer): similar to normalization but it instead uses the interquartile range, so that it is robust to outliers.

In [24]:
from sklearn.preprocessing import QuantileTransformer

qt = QuantileTransformer(output_distribution='normal')

In [37]:
recsys_df['score_language_saber_11'] = qt.fit_transform(recsys_df[['score_language_saber_11']].values.reshape(-1,1))
recsys_df['score_mathematics_saber_11'] = qt.fit_transform(recsys_df[['score_mathematics_saber_11']].values.reshape(-1,1))
recsys_df['score_biology_saber_11'] = qt.fit_transform(recsys_df[['score_biology_saber_11']].values.reshape(-1,1))
recsys_df['score_chemistry_saber_11'] = qt.fit_transform(recsys_df[['score_chemistry_saber_11']].values.reshape(-1,1))
recsys_df['score_physics_saber_11'] = qt.fit_transform(recsys_df[['score_physics_saber_11']].values.reshape(-1,1))
recsys_df['score_social_science_saber_11'] = qt.fit_transform(recsys_df[['score_social_science_saber_11']].values.reshape(-1,1))
recsys_df['score_philosophy_saber_11'] = qt.fit_transform(recsys_df[['score_philosophy_saber_11']].values.reshape(-1,1))
recsys_df['score_english_saber_11'] = qt.fit_transform(recsys_df[['score_english_saber_11']].values.reshape(-1,1))

In [38]:
for var in features:
    print(var.upper() + ' Distribution')
    print(recsys_df[var].describe())
    print('-----')

In [39]:
for cat in features:
    cat_num = recsys_df[cat]
    print("Plot for %s: total counts = %d" % (cat.upper(), len(cat_num)))
    f, ax = plt.subplots(figsize=(10,5))
    plot = sns.distplot(cat_num)
    plot.set_xticklabels(plot.get_xticklabels(), rotation=45)
    plt.show()

In [40]:
#create a mapping from labels to a unique integer and vice versa for labelling and prediction later
labels = recsys_df['your_undergraduate_core'].unique()
i = 0
idx2class = {} 
class2idx = {}
for tp in labels:
    idx2class[i] = tp
    class2idx[tp] = i
    i += 1
idx2class

In [41]:
class2idx

In [31]:
features_train = [
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
      # target
     'your_undergraduate_core'
]


In [46]:
recsys = recsys_df[features_train]
recsys.head()

In [47]:
recsys['label'] = recsys['your_undergraduate_core'].replace(class2idx)

In [48]:
recsys.head()

When a dataset is imbalanced, i.e., the distribution of target variables is not uniform, we should make sure that the training-validation split is stratified. This ensures that the distribution of the target variable is preserved in both the training and validation datasets.

In [49]:
def stratified_split(df, target, val_percent=0.2):
    '''
    Function to split a dataframe into train and validation sets, while preserving the ratio of the labels in the target variable
    Inputs:
    - df, the dataframe
    - target, the target variable
    - val_percent, the percentage of validation samples, default 0.2
    Outputs:
    - train_idxs, the indices of the training dataset
    - val_idxs, the indices of the validation dataset
    '''
    classes=list(df[target].unique())
    train_idxs, val_idxs = [], []
    for c in classes:
        idx=list(df[df[target]==c].index)
        np.random.shuffle(idx)
        val_size=int(len(idx)*val_percent)
        val_idxs+=idx[:val_size]
        train_idxs+=idx[val_size:]
    return train_idxs, val_idxs

In [50]:
train_idxs, val_idxs = stratified_split(recsys, 'label', val_percent=0.25)

val_idxs, test_idxs = stratified_split(recsys[recsys.index.isin(val_idxs)], 'label', val_percent=0.5)

In [51]:
def test_stratified(df, col):
    '''
    Analyzes the ratio of different classes in a categorical variable within a dataframe
    Inputs:
    - dataframe
    - categorical column to be analyzed
    Returns: None
    '''
    classes=list(df[col].unique())
    
    for c in classes:
        print(f'Proportion of records with {c}: {len(df[df[col]==c])*1./len(df):0.2} ({len(df[df[col]==c])} / {len(df)})')

In [52]:
print('---------- STRATIFIED SAMPLING REPORT ----------')
print('-------- Label proportions in FULL data --------')
test_stratified(recsys, 'label')
print('-------- Label proportions in TRAIN data --------')
test_stratified(recsys[recsys.index.isin(train_idxs)], 'label')
print('------ Label proportions in VALIDATION data -----')
test_stratified(recsys[recsys.index.isin(val_idxs)], 'label')
print('-------- Label proportions in TEST data ---------')
test_stratified(recsys[recsys.index.isin(test_idxs)], 'label')

In [53]:
train_df = recsys[recsys.index.isin(train_idxs)]
X_train = train_df[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_train = train_df[['label']].values
print('Retrieved Training Data')
val_df = recsys[recsys.index.isin(val_idxs)]
X_val = val_df[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_val = val_df[['label']].values
print('Retrieved Validation Data')
test_df = recsys[recsys.index.isin(test_idxs)]
X_test = test_df[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_test = test_df[['label']].values
print('Retrieved Test Data')

In [64]:
recsys['label'].value_counts()

# Model Building

For simplicity, I will explore Logistic regression(OneVsRest) as the baseline model and progressively use more complicated ensemble models for our prediction task such as XGBoost.

In [54]:
from sklearn.model_selection import GridSearchCV
#store data, all in numpy arrays
training_data = {'X_train':X_train,'Y_train':Y_train,
                'X_val': X_val,'Y_val':Y_val,
                'X_test': X_test,'Y_test':Y_test}

## Random Forest¶

In [55]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_jobs=None,random_state=27,
                       verbose=1)
clf.fit(training_data['X_train'], training_data['Y_train'].reshape(training_data['Y_train'].shape[0],))

In [56]:
predicted_labels = clf.predict(training_data['X_test'])

In [57]:
accuracy_score(training_data['Y_test'], predicted_labels)

In [62]:
from sklearn.metrics import classification_report
print(classification_report(training_data['Y_test'], predicted_labels))

In [68]:
import xgboost as xgb
# allow logloss and classification error plots for each iteraetion of xgb model
def plot_compare(metrics,eval_results,epochs):
    for m in metrics:
        test_score = eval_results['val'][m]
        train_score = eval_results['train'][m]
        rang = range(0, epochs)
        plt.rcParams["figure.figsize"] = [6,6]
        plt.plot(rang, test_score,"c", label="Val")
        plt.plot(rang, train_score,"orange", label="Train")
        title_name = m + " plot"
        plt.title(title_name)
        plt.xlabel('Iterations')
        plt.ylabel(m)
        lgd = plt.legend()
        plt.show()
        
def fitXgb(sk_model, training_data=training_data,epochs=300):
    print('Fitting model...')
    sk_model.fit(training_data['X_train'], training_data['Y_train'].reshape(training_data['Y_train'].shape[0],))
    print('Fitting done!')
    train = xgb.DMatrix(training_data['X_train'], label=training_data['Y_train'])
    val = xgb.DMatrix(training_data['X_val'], label=training_data['Y_val'])
    params = sk_model.get_xgb_params()
    metrics = ['mlogloss','merror']
    params['eval_metric'] = metrics
    store = {}
    evallist = [(val, 'val'),(train,'train')]
    xgb_model = xgb.train(params, train, epochs, evallist,evals_result=store,verbose_eval=100)
    print('-- Model Report --')
    print('XGBoost Accuracy: '+str(accuracy_score(sk_model.predict(training_data['X_test']), training_data['Y_test'])))
    print('XGBoost F1-Score (Micro): '+str(f1_score(sk_model.predict(training_data['X_test']),training_data['Y_test'],average='micro')))
    plot_compare(metrics,store,epochs)
    features = ['score_language_saber_11', 'score_mathematics_saber_11',
      'score_biology_saber_11',  'score_chemistry_saber_11',
      'score_physics_saber_11',  'score_social_science_saber_11',
      'score_philosophy_saber_11',  'score_english_saber_11']
    f, ax = plt.subplots(figsize=(10,5))
    plot = sns.barplot(x=features, y=sk_model.feature_importances_)
    ax.set_title('Feature Importance')
    plot.set_xticklabels(plot.get_xticklabels(),rotation='vertical')
    plt.show()

### XGBoost hyperparameter tuning with GridSearchCV
#### Initial model

In [72]:
from xgboost.sklearn import XGBClassifier
#initial model
xgb1 = XGBClassifier(learning_rate=0.1,
                    n_estimators=500,
                    max_depth=5,
                    min_child_weight=1,
                    gamma=0,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    #objective='multi:softmax',
                    nthread=4,
                    tree_method='gpu_hist',
                    num_class=64,
                    seed=27)

In [73]:
%%time
fitXgb(xgb1, training_data)